In [3]:
from astropy.io import fits
import os
import os.system
import shutil
import time

startTime = time.clock()

regressionPath = '/Users/jnoss/dev/regression/regress'
execPath = '/Users/jnoss/dev/builds/master'
outPath = '/Users/jnoss/dev/regression/master/'

print('Path containing test data: "{0}"'.format(regressionPath))
print('Path containing executables: "{0}"'.format(execPath))
print('Path to dump all output: "{0}"'.format(outPath))
print('\n')

cteOnly = True
compareOnTheFly = False
compareOnly = False

#create output dir and/or move old one to outPath + '-old'
os.mkdir(outPath)
logPath = os.path.join(outPath, 'logs')
os.mkdir(logPath)

wf3Input = []
acsInput = []
stisInput = []

tree = os.walk(regressionPath, topdown = True, followlinks = False)
for root, dir, files in tree:
    for fname in files:
        if 'raw.fits' in fname and 'raw.fits.svn-base' not in fname:
            fullFilePath = os.path.join(root, fname)
            hdu = fits.open(fullFilePath)
            instrument = hdu[0].header['instrume']
            hdu.close()
            if 'WFC3' in instrument:
                wf3Input.append(fullFilePath)
            elif 'ACS' in instrument:
                acsInput.append(fullFilePath)
            elif 'STIS' in instrument:
                stisInput.append(fullFilePath)

print('{0} acs input files found.'.format(str(len(acsInput))))
print('{0} stis input files found.'.format(str(len(stisInput))))
print('{0} wf3 input files found.\n'.format(str(len(wf3Input))))

print('Time taken to walk directory tree: {0}\n'.format(str(time.clock() - startTime)))

if cteOnly:
    print('Processing CTE corrections only:\n')

    wf3CTEInput = []
    for entry in wf3Input:
        hdu = fits.open(entry)
        try:
            cteCorrect = hdu[0].header['PCTECORR']
        except KeyError:
            #print('Missing "PCTECORR" keyword, skipping ' + entry)
            continue
        hdu.close()
        if 'PERFORM' in cteCorrect:
            wf3CTEInput.append(entry)
    
    print('{0} wf3cte input files found.\n'.format(str(len(wf3CTEInput))))
    
    cmd = os.path.join(execPath, 'wf3cte.e')
    if not os.exits(cmd):
        print('ERROR: The required executable "{0}" does not exist!'.format(cmd))
        quit()
    
    #Acquire version
    version = subprocess.run([cmd, '--version'], shell=True, check=False, stdout=subprocess.PIPE).stdout
       
    os.chdir(outPath)
    for entry in wf3CTEInput:
        print('Processing "{0}"...'.format(entry)
        #Run wf3cte on relevant files (one at a ime)
        results = subprocess.run(['time', cmd, '-v', entry], shell=True, check=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        #Now look at results and write them to a log file in outPath
        logFile = os.path.basename(entry) + '.log'
        logFile = os.path.join(logPath, logFile)
        try:
            fid = open(logFile)
        except OSError as err:
            print('ERROR: Cannot write log file "{0}" due to:'.format(logFile))
            print('"{0}"'.format(err))
        else:
            fid.write('Input file: "{0}"\n'.format(entry))
            fid.write('Program: "{0}"\n'.format(cmd))
            fid.write('Version: {0}\n'.format(str(version)))
            fid.write('return code:{0}\n\n'.format(str(results.return_code)))
            fid.write('stdout results:\n')
            fid.write(results.stdout)
            fid.write('\n\n')
            fid.write('stderr results:\n')
            fid.write(results.stderr)
            fid.write('\n\n')
            fid.close()
        
        if results.returncode:
            print('Process failed')
        else:
            print('Process succeded')
            
            #Check each test on the fly
            if compareOnTheFly:
                print('Comparing results...')
                #do comparison with FITSDiff
            
            
        
    print('Time taken: '+ str(time.clock() - startTime))
    quit()



SyntaxError: invalid syntax (<ipython-input-3-97a1d1bb66fc>, line 81)

In [ ]:
rawfile = fits.open('dev/test/cte/wf3/truth/ic0q01ynq_raw.fits')
truthfile = fits.open('dev/test/cte/wf3/truth/ic0q01ynq_rac_tmp.fits')
devfile = fits.open('dev/test/cte/wf3/ic0q01ynq_rac_tmp.fits')

diff = fits.FITSDiff(devfile, truthfile, ignore_keywords=['date'])
diff.identical

pixDiff = fits.ImageDataDiff(rawfile[1].data, truthfile[1].data, tolerance=0.0, numdiffs=10)
print(pixDiff.identical)
pixDiff.diff_pixels
#diff.report()